In [1]:
using Random
using Printf
using StatsBase
using DelimitedFiles
using Symbolics
using NLsolve
using LinearAlgebra

In [10]:
# stability check - model with self-anticonformity
function jacobian_2025()
    @variables uu ud du dd α p he hp q

    # Define the symbolic system equations
    F1 = uu*(1 - α*p/2 - (1-α)*(1-p)*(1-he)*(1-uu-ud)^q) +
        ud*α*(p/2 + (1-p)*hp + (1-p)*(1-hp)*(uu+ud)^q) +
        du*(1-α)*(1 - (1-p)*(1-he)*(1-uu-ud)^q) #- uu

    F2 = uu*α*p/2 +
        ud*((1-α)*(1-p)*(he + (1-he)*(uu+ud)^q) + α*(1-p/2) - α*(1-p)*(hp + (1-hp)*(uu+ud)^q)) +
        dd*(1-α)*(1-p)*(he + (1-he)*(uu+ud)^q) #- ud

    F3 = uu*(1-α)*(1-p)*(1-he)*(1-uu-ud)^q +
        du*((1-α)*(1-p)*(1-he)*(1-uu-ud)^q + α*(1 - p/2 - (1-p)*hp - (1-p)*(1-hp)*(1-uu-ud)^q)) +
        dd*α*(p/2 + (1-p)*hp) # - du

    F4 = 1 - (F1 + F2 + F3)

    # Create the system vector and variable vector
    F_vec = [F1, F2, F3, F4]
    vars = [uu, ud, du, dd]

    # Calculate the symbolic Jacobian
    # J_symbolic = Symbolics.jacobian(F_vec, vars)


    J = Symbolics.jacobian(F_vec, vars)

    # === Optional: simplify the Jacobian ===
    J_simplified = simplify.(J)
    println("Symbolic Jacobian: ")  
    println(J_simplified)
    return J_simplified

end


function checkstability(alpha_, p_, he_, hp_, q_, uu_, ud_, du_, dd_, jacobian)
    @variables uu ud du dd α p he hp q

    # Substitute fixed point + parameters into symbolic Jacobian
    subs_dict = Dict(α => alpha_, p => p_, he => he_, hp => hp_, q => q_,
        uu => uu_, ud => ud_,
        du => du_, dd => dd_)

    numericjacobian = substitute(jacobian, subs_dict) |> eval

    println("Numeric Jacobian: ")
    println(numericjacobian)

    # Calculate eigenvalues
    eigenvalues = eigvals(numericjacobian)
    println("Eigenvalues: ", eigenvalues)

    # Stability analysis
    if all(abs.(eigenvalues) .< 1)
        println("Fixed point is STABLE")
        return true
    else
        println("Fixed point is UNSTABLE")
        return false
    end
    
end


checkstability (generic function with 1 method)

In [12]:
yearvariant = "2025"
q = 3
alpha = 0.1

he = 0
hp = 0.
# h = [[0, 0], [0.01, 0], [0.05, 0], [0, 0.01], [0, 0.05]]

if yearvariant == "2025"
    jacobian_func = jacobian_2025()
    println("Jacobian for 2025 done")
elseif yearvariant == "2018"
    jacobian_func = jacobian_2018()
    println("Jacobian for 2018 done")
else 
    error("Unknown year variant: $yearvariant")
end

Symbolic Jacobian: 
Num[1 - (1//2)*p*α + (-1 + he)*(1 - p)*((1 - ud - uu)^q)*(1 - α) - du*(-1 + he)*(1 - p)*q*((1 - ud - uu)^(-1 + q))*(1 - α) - (-1 + he)*(1 - p)*q*uu*((1 - ud - uu)^(-1 + q))*(1 - α) + (1 - hp)*(1 - p)*q*ud*((ud + uu)^(-1 + q))*α ((1//2)*p + hp*(1 - p) + (1 - hp)*(1 - p)*((ud + uu)^q))*α - du*(-1 + he)*(1 - p)*q*((1 - ud - uu)^(-1 + q))*(1 - α) - (-1 + he)*(1 - p)*q*uu*((1 - ud - uu)^(-1 + q))*(1 - α) + (1 - hp)*(1 - p)*q*ud*((ud + uu)^(-1 + q))*α (1 + (-1 + he)*(1 - p)*((1 - ud - uu)^q))*(1 - α) 0; (1//2)*p*α + dd*(1 - he)*(1 - p)*q*((ud + uu)^(-1 + q))*(1 - α) + ((1 - he)*(1 - p)*q*((ud + uu)^(-1 + q))*(1 - α) + (1 - hp)*(-1 + p)*q*((ud + uu)^(-1 + q))*α)*ud (1 - (1//2)*p)*α + (he + (1 - he)*((ud + uu)^q))*(1 - p)*(1 - α) + (hp + (1 - hp)*((ud + uu)^q))*(-1 + p)*α + dd*(1 - he)*(1 - p)*q*((ud + uu)^(-1 + q))*(1 - α) + ((1 - he)*(1 - p)*q*((ud + uu)^(-1 + q))*(1 - α) + (1 - hp)*(-1 + p)*q*((ud + uu)^(-1 + q))*α)*ud 0 (he + (1 - he)*((ud + uu)^q))*(1 - p)*(1 - α); (1 

In [13]:
pathanalytics = "C:/Users/Basia/OneDrive - Politechnika Wroclawska/Pulpit/PhD/Current work/ExternalInfluence/MFA"
file = "EPO_$(yearvariant)_q$(q)_alpha_$(@sprintf("%.2f", alpha))_h_e_$(@sprintf("%.2f", he))_h_p_$(@sprintf("%.2f", hp))_4vars.txt"


# Load each file into a DataFrame
df_ = readdlm(joinpath(pathanalytics, file), '\t')
# append a column of zeros
df_ = hcat(df_, zeros(size(df_, 1)))
# println(df_[1:5, :])

for rowid in eachindex(df_[:, 1])
    p_temp = df_[rowid, 1]
    uu_temp = df_[rowid, 2]
    ud_temp = df_[rowid, 3]
    du_temp = df_[rowid, 4]
    dd_temp = df_[rowid, 5]

    if checkstability(alpha, p_temp, he, hp, q, uu_temp, ud_temp, du_temp, dd_temp, jacobian_func)
        df_[rowid, 6] = 1.0
    else
        df_[rowid, 6] = 0.0
    end

end

println("Stability check done.")
# save the updated dataframe
# exportwithstability(file, df_, yearvariant)

println(df_[1:5, :])



Numeric Jacobian: 
Real[0.09999999999999998 0.0 0.0 0; 0.0 0.1 0 0.0; 0.9 -0.0 0.9 0.0; -1.0 -0.1 -0.9 0.0]
Eigenvalues: [0.0, 0.09999999999999998, 0.1, 0.9]
Fixed point is STABLE
Numeric Jacobian: 
Real[1.22962 0.35462000000000005 0.7875 0; 0.33288 0.5328800000000001 0 0.1125; -0.22038000000000002 -0.33288 0.2 0.0; -1.34212 -0.55462 -0.9875 -0.1125]
Eigenvalues: [2.4801154827063338e-17, 0.08750000000000008, 0.7201178725393341, 1.0423821274606646]
Fixed point is UNSTABLE
Numeric Jacobian: 
Real[1.0 0.1 0.9 0; 0.0 0.9 0 0.9; 0.0 0.0 0.1 0.0; -1.0 -1.0 -1.0 -0.9]
Eigenvalues: [-7.905326584942494e-15, 0.1, 0.1000000000000077, 0.9000000000000006]
Fixed point is STABLE
Numeric Jacobian: 
Real[0.10219864999999995 0.00139865 0.0009000000000000008 0; 5.0e-5 0.09995000000000001 0 0.0; 0.8979012 -0.0011988 0.89915 5.0e-5; -1.00014985 -0.10014985000000001 -0.90005 -5.0e-5]
Eigenvalues: [1.6714787165252944e-18, 0.0998958079232336, 0.10124011887082959, 0.9001127232059369]
Fixed point is STABLE
Nume

Excessive output truncated after 524307 bytes.

Eigenvalues: [-5.285220554486967e-16, 0.02178750000000061, 0.2624889497869594, 0.9273735502130416]
Fixed point is STABLE
Numeric Jacobian: 
Real[1.0188230999999999 0.12502310000000003 0.8721 0; 0.12067690000000002 0.17027690000000004 0 0.0279; -0.055176900000000015 -0.08307690000000001 0.0872 0.0376; -1.0843231000000002 -0.21222310000000005 -0.9593 -0.0655]
Eigenvalues: [-5.775960294312274e-17, 0.021700000000000306, 0.2618396386644984, 0.9272603613355016]
Fixed point is STABLE
Numeric Jacobian: 
Real[1.0185437350000002 0.124718735 0.8722125000000001 0; 0.12039376500000001 0.16979376499999999 0 0.0277875; 